### 6. Embedding(DNN Model Training and Test)

In [1]:
# import libraries used

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import  OneHotEncoder, LabelEncoder

import swifter
import gc

In [2]:
import tensorflow as tf
import tensorflow
from tensorflow import keras

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import  OneHotEncoder, LabelEncoder

In [3]:
# Load processed dataset.

data = pd.read_pickle('v30_data.pkl')

In [4]:
# train, validation, test data set parameter setting.

test_date_block = 34
val_date_block = test_date_block - 1
train_date_block = val_date_block - 1

#### 6.1 Embedding Model Training

#### 6.1.1 All Samples Model

In [5]:
on_sale_data = data.copy()

In [6]:
drop_list_on_sale = [
    'days',
    'shop_city',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    'shop_item_on_market',
    #'shop_item_no_sale',
    #'avg_date_item_price_cnt_lag_1',
    'avg_date_item_price_cnt_lag_2',
    'avg_date_item_price_cnt_lag_3',
    #'avg_date_shop_item_price_cnt_lag_1',
    'avg_date_shop_item_price_cnt_lag_2',
    'avg_date_shop_item_price_cnt_lag_3',
    'avg_date_cat_price_cnt_lag_1',
    'avg_date_cat_price_cnt_lag_2',
    'avg_date_cat_price_cnt_lag_3',
    'avg_date_shop_cat_price_cnt_lag_1',
    'avg_date_shop_cat_price_cnt_lag_2',
    'avg_date_shop_cat_price_cnt_lag_3',
    'date_item_price_lag_2',
    'date_item_price_lag_3',
    'date_shop_item_price_lag_2',
    'date_shop_item_price_lag_3',
    #'date_shop_revenue_lag_1',
    'date_shop_revenue_lag_2',
    'date_shop_revenue_lag_3',
    #'date_shop_cat_revenue_lag_1',
    'date_shop_cat_revenue_lag_2',
    'date_shop_cat_revenue_lag_3',
]

on_sale_data.drop(drop_list_on_sale, axis=1, inplace=True)

In [7]:
#just extract the samples within the test dataset
'''
shop_list = list(on_sale_data[on_sale_data.date_block_num == test_date_block].shop_id.unique())
item_list = list(on_sale_data[on_sale_data.date_block_num == test_date_block].item_id.unique())

on_sale_data = on_sale_data[on_sale_data.shop_id.isin(shop_list)]
on_sale_data = on_sale_data[on_sale_data.item_id.isin(item_list)]
'''

'\nshop_list = list(on_sale_data[on_sale_data.date_block_num == test_date_block].shop_id.unique())\nitem_list = list(on_sale_data[on_sale_data.date_block_num == test_date_block].item_id.unique())\n\non_sale_data = on_sale_data[on_sale_data.shop_id.isin(shop_list)]\non_sale_data = on_sale_data[on_sale_data.item_id.isin(item_list)]\n'

In [8]:
#on_sale_data_ft = on_sale_data[on_sale_data.time_on_market > 0]
on_sale_data_ft = on_sale_data.copy()

In [9]:
X_train_on_sale = on_sale_data_ft[(on_sale_data_ft.date_block_num <= train_date_block)].drop(['date_shop_item_cnt'], axis=1)
Y_train_on_sale = on_sale_data_ft[(on_sale_data_ft.date_block_num <= train_date_block)]['date_shop_item_cnt']
X_valid_on_sale = on_sale_data_ft[(on_sale_data_ft.date_block_num == val_date_block)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_on_sale = on_sale_data_ft[(on_sale_data_ft.date_block_num == val_date_block)]['date_shop_item_cnt']

In [10]:
trainX = []
trainY = np.array(Y_train_on_sale)
trainX.append(np.array(X_train_on_sale['date_block_num']))
trainX.append(np.array(X_train_on_sale['shop_id']))
#trainX.append(np.array(X_train_on_sale['shop_city']))
trainX.append(np.array(X_train_on_sale['shop_type']))
trainX.append(np.array(X_train_on_sale['item_category_id']))
trainX.append(np.array(X_train_on_sale['cat_type']))
trainX.append(np.array(X_train_on_sale['sub_type']))
trainX.append(np.array(X_train_on_sale['month']))

#trainX.append(np.array(X_train_on_sale.drop(['date_block_num','shop_id','shop_city','shop_type',\
#        'item_category_id','cat_type','sub_type','month'],axis=1)))
trainX.append(np.array(X_train_on_sale.drop(['date_block_num','shop_id','shop_type',\
        'item_category_id','cat_type','sub_type','month'],axis=1)))

validX = []
validY = np.array(Y_valid_on_sale)
validX.append(np.array(X_valid_on_sale['date_block_num']))
validX.append(np.array(X_valid_on_sale['shop_id']))
#validX.append(np.array(X_valid_on_sale['shop_city']))
validX.append(np.array(X_valid_on_sale['shop_type']))
validX.append(np.array(X_valid_on_sale['item_category_id']))
validX.append(np.array(X_valid_on_sale['cat_type']))
validX.append(np.array(X_valid_on_sale['sub_type']))
validX.append(np.array(X_valid_on_sale['month']))

#validX.append(np.array(X_valid_on_sale.drop(['date_block_num','shop_id','shop_city','shop_type',\
#        'item_category_id','cat_type','sub_type','month'],axis=1)))
validX.append(np.array(X_valid_on_sale.drop(['date_block_num','shop_id','shop_type',\
        'item_category_id','cat_type','sub_type','month'],axis=1)))

In [11]:
output_dim_date_block_num = 6
output_dim_shop_id = 10
#output_dim_shop_city = 5
output_dim_shop_type = 3
output_dim_item_cat = 10
output_dim_cat_type = 4
output_dim_sub_type = 8
output_dim_month = 4

input_dim = len(X_train_on_sale.columns)
input_dim_non_category = input_dim - 7

input_dim_date_block_num = on_sale_data_ft.date_block_num.max() + 1
input_dim_shop_id = on_sale_data_ft.shop_id.max() + 1
#input_dim_shop_city = on_sale_data_ft.shop_city.max() + 1
input_dim_shop_type = on_sale_data_ft.shop_type.max() + 1
input_dim_item_cat = on_sale_data_ft.item_category_id.max() + 1
input_dim_cat_type = on_sale_data_ft.cat_type.max() + 1
input_dim_sub_type = on_sale_data_ft.sub_type.max() + 1
input_dim_month = on_sale_data_ft.month.max() + 1


def build_embedding_model():
    inputs = []
    embeddings = []
    
    date_block_num_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_date_block_num, output_dim_date_block_num, input_length=1, name='date_block_num')(date_block_num_feature)
    embedding = Reshape(target_shape=(output_dim_date_block_num,))(embedding)
    inputs.append(date_block_num_feature)
    embeddings.append(embedding)
    
    shop_id_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_shop_id, output_dim_shop_id, input_length=1, name='shop_id')(shop_id_feature)
    embedding = Reshape(target_shape=(output_dim_shop_id,))(embedding)
    inputs.append(shop_id_feature)
    embeddings.append(embedding)
    
    '''
    shop_city_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_shop_city, output_dim_shop_city, input_length=1, name='shop_city')(shop_city_feature)
    embedding = Reshape(target_shape=(output_dim_shop_city,))(embedding)
    inputs.append(shop_city_feature)
    embeddings.append(embedding)
    '''
    
    shop_type_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_shop_type, output_dim_shop_type, input_length=1, name='shop_type')(shop_type_feature)
    embedding = Reshape(target_shape=(output_dim_shop_type,))(embedding)
    inputs.append(shop_type_feature)
    embeddings.append(embedding)
    
    item_cat_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_item_cat, output_dim_item_cat, input_length=1, name='item_category_id')(item_cat_feature)
    embedding = Reshape(target_shape=(output_dim_item_cat,))(embedding)
    inputs.append(item_cat_feature)
    embeddings.append(embedding)
    
    cat_type_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_cat_type, output_dim_cat_type, input_length=1, name='cat_type')(cat_type_feature)
    embedding = Reshape(target_shape=(output_dim_cat_type,))(embedding)
    inputs.append(cat_type_feature)
    embeddings.append(embedding)
    
    sub_type_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_sub_type, output_dim_sub_type, input_length=1, name='sub_type')(sub_type_feature)
    embedding = Reshape(target_shape=(output_dim_sub_type,))(embedding)
    inputs.append(sub_type_feature)
    embeddings.append(embedding)
    
    month_feature = Input(shape=(1,))
    embedding = Embedding(input_dim_month, output_dim_month, input_length=1, name='month')(month_feature)
    embedding = Reshape(target_shape=(output_dim_month,))(embedding)
    inputs.append(month_feature)
    embeddings.append(embedding)
    
    non_category_input = Input(shape=(input_dim_non_category,))
    inputs.append(non_category_input)
    embeddings.append(non_category_input)
    
    x = keras.layers.Concatenate()(embeddings)
    x = Dense(1024,input_shape=(input_dim,),activation='relu')(x)
    x = Dense(512,input_shape=(input_dim,),activation='relu')(x)
    output = Dense(1)(x)
    
    model = Model(inputs, output)
    
    model.compile(loss="mse", optimizer="Adam", metrics=[RootMeanSquaredError()])
    
    return model

In [12]:
num_epochs = 1

embedding_model = build_embedding_model()    
history = embedding_model.fit(trainX, trainY, validation_data=(validX, validY), \
              epochs=num_epochs, batch_size=100)

2021-11-28 21:49:10.366955: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-28 21:49:10.368302: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Train on 10674594 samples, validate on 238172 samples
10674594/10674594 [==============================] - 1634s 153us/sample - loss: 308.3928 - root_mean_squared_error: 17.5526 - val_loss: 0.8774 - val_root_mean_squared_error: 0.9367


#### 6.2 Categorical Feature Extraction

In [13]:
# get embedding of date_block_num

date_block_num_model = Model(inputs = embedding_model.get_layer("date_block_num").input,\
                outputs = embedding_model.get_layer("date_block_num").output)
date_block_num_list = []
for x in set(data.date_block_num):
    embedding = date_block_num_model.predict(np.array([x])).reshape(output_dim_date_block_num, )
    date_block_num_list.append([x] + list(embedding))

date_block_num_columns = ['date_block_num']
for i in range(output_dim_date_block_num):
    date_block_num_columns.append("date_block_num_" + str(i))

embed_date_block_num = pd.DataFrame(date_block_num_list)
embed_date_block_num.columns = date_block_num_columns
embed_date_block_num

,date_block_num,date_block_num_0,date_block_num_1,date_block_num_2,date_block_num_3,date_block_num_4,date_block_num_5
0,0,1.268768,0.322647,1.171116,-1.314244,1.202230,-1.045298
1,1,-0.532079,1.246603,-0.300013,-0.136824,-0.516934,-0.565331
2,2,0.274711,0.691531,0.415588,-0.622441,0.330729,-0.540563
3,3,-1.404411,-1.773321,-1.487296,1.434112,-1.464543,1.728903
4,4,-0.846421,-0.937722,-0.856397,0.819879,-0.891456,0.977206
5,5,-0.051947,0.139767,-0.136289,-0.072773,-0.137388,0.010660
6,6,-0.388418,0.016723,-0.412561,0.199834,-0.422537,0.160726
7,7,-0.001739,0.621340,0.032203,-0.299143,-0.016441,-0.297646
8,8,0.127135,0.170997,0.079640,-0.260836,0.052109,-0.096713
9,9,-0.347297,-0.529249,-0.331852,0.273935,-0.350546,0.474521


In [14]:
# get embedding of shop_id

shop_id_model = Model(inputs = embedding_model.get_layer("shop_id").input,\
                outputs = embedding_model.get_layer("shop_id").output)
shop_id_list = []
for x in set(data.shop_id):
    embedding = shop_id_model.predict(np.array([x])).reshape(output_dim_shop_id, )
    shop_id_list.append([x] + list(embedding))

shop_id_columns = ['shop_id']
for i in range(output_dim_shop_id):
    shop_id_columns.append("shop_id_" + str(i))

embed_shop_id = pd.DataFrame(shop_id_list)
embed_shop_id.columns = shop_id_columns
embed_shop_id

,shop_id,shop_id_0,shop_id_1,shop_id_2,shop_id_3,shop_id_4,shop_id_5,shop_id_6,shop_id_7,shop_id_8,shop_id_9
0,2,0.909650,0.977599,-0.912823,-0.760737,-0.908641,-1.061325,-0.893207,0.838867,-0.940534,-0.926699
1,3,0.859641,0.864575,-0.819124,-0.614846,-0.798228,-1.077686,-0.842881,0.749784,-0.763402,-0.823279
2,4,0.753621,0.618335,-0.560332,-0.637453,-0.688511,-0.715357,-0.631611,0.542721,-0.547243,-0.726650
3,5,0.670417,0.614260,-0.765129,-0.567666,-0.607773,-0.879605,-0.604879,0.634527,-0.606199,-0.607831
4,6,0.163101,0.144872,-0.120184,-0.188384,-0.095765,-0.195437,-0.148562,0.021801,0.033907,-0.110240
5,7,0.346429,0.330380,-0.249192,-0.371657,-0.257925,-0.387678,-0.363972,0.251782,-0.315657,-0.363700
6,8,1.852101,1.749307,-1.771764,-1.724198,-1.726369,-1.895647,-1.795547,1.783551,-1.772643,-1.782899
7,9,-1.461282,-1.472036,1.534742,1.093978,1.480527,1.416363,1.409558,-1.582807,1.580619,1.357563
8,10,1.073761,0.995890,-1.078100,-0.943145,-1.018437,-1.272919,-0.983736,1.006871,-1.048171,-1.040043
9,12,0.698817,0.645229,-0.597431,-0.711697,-0.495004,-0.894996,-0.594185,0.538854,-0.465191,-0.667776


In [15]:
# get embedding of shop_city
'''
shop_city_model = Model(inputs = embedding_model.get_layer("shop_city").input,\
                outputs = embedding_model.get_layer("shop_city").output)
shop_city_list = []
for x in set(data.shop_city):
    embedding = shop_city_model.predict(np.array([x])).reshape(output_dim_shop_city, )
    shop_city_list.append([x] + list(embedding))

shop_city_columns = ['shop_city']
for i in range(output_dim_shop_city):
    shop_city_columns.append("shop_city_" + str(i))

embed_shop_city = pd.DataFrame(shop_city_list)
embed_shop_city.columns = shop_city_columns
embed_shop_city
'''

'\nshop_city_model = Model(inputs = embedding_model.get_layer("shop_city").input,                outputs = embedding_model.get_layer("shop_city").output)\nshop_city_list = []\nfor x in set(data.shop_city):\n    embedding = shop_city_model.predict(np.array([x])).reshape(output_dim_shop_city, )\n    shop_city_list.append([x] + list(embedding))\n\nshop_city_columns = [\'shop_city\']\nfor i in range(output_dim_shop_city):\n    shop_city_columns.append("shop_city_" + str(i))\n\nembed_shop_city = pd.DataFrame(shop_city_list)\nembed_shop_city.columns = shop_city_columns\nembed_shop_city\n'

In [16]:
# get embedding of shop_type

shop_type_model = Model(inputs = embedding_model.get_layer("shop_type").input,\
                outputs = embedding_model.get_layer("shop_type").output)
shop_type_list = []
for x in set(data.shop_type):
    embedding = shop_type_model.predict(np.array([x])).reshape(output_dim_shop_type, )
    shop_type_list.append([x] + list(embedding))

shop_type_columns = ['shop_type']
for i in range(output_dim_shop_type):
    shop_type_columns.append("shop_type_" + str(i))

embed_shop_type = pd.DataFrame(shop_type_list)
embed_shop_type.columns = shop_type_columns
embed_shop_type

,shop_type,shop_type_0,shop_type_1,shop_type_2
0,0,0.193707,0.886832,0.277056
1,1,-0.098244,-3.572457,-0.513338
2,2,-0.244678,0.842488,0.050010
3,4,0.168824,0.375227,0.310833
4,5,-0.338635,-0.746723,-0.463173
5,6,0.233995,0.377423,0.301347
6,7,0.265973,-0.579477,0.192233
7,8,1.665980,-0.625718,1.615868


In [17]:
# get embedding of item_cat

item_cat_model = Model(inputs = embedding_model.get_layer("item_category_id").input,\
                outputs = embedding_model.get_layer("item_category_id").output)
item_cat_list = []
for x in set(data.item_category_id):
    embedding = item_cat_model.predict(np.array([x])).reshape(output_dim_item_cat, )
    item_cat_list.append([x] + list(embedding))

item_cat_columns = ['item_category_id']
for i in range(output_dim_item_cat):
    item_cat_columns.append("item_cat_" + str(i))

embed_item_cat = pd.DataFrame(item_cat_list)
embed_item_cat.columns = item_cat_columns
embed_item_cat

,item_category_id,item_cat_0,item_cat_1,item_cat_2,item_cat_3,item_cat_4,item_cat_5,item_cat_6,item_cat_7,item_cat_8,item_cat_9
0,0,1.253417,-0.830337,1.348427,1.317632,-1.286261,1.263842,-1.301289,1.305832,-1.386951,-1.414681
1,1,1.142443,-0.960683,1.083402,1.041896,-1.111754,1.064523,-1.121771,1.155054,-1.053843,-1.081777
2,2,0.270019,-0.711962,0.238076,0.081517,-0.149379,0.089047,-0.265474,0.106642,-0.295984,-0.304303
3,3,0.276170,-0.269644,0.307893,0.312854,-0.368024,0.210699,-0.383182,0.425412,-0.630769,-0.361592
4,4,0.506199,-1.021186,0.540130,0.462938,-0.446148,0.452638,-0.426494,0.374380,-0.414989,-0.474865
...,...,...,...,...,...,...,...,...,...,...,...
79,79,-1.834332,-0.038536,-1.706199,-1.433592,1.406276,-1.470885,1.730538,-1.397059,0.609196,1.654122
80,80,-0.970178,-0.755545,-0.945334,-0.656822,0.741162,-0.876144,0.598279,-0.651716,-0.403101,0.817427
81,81,0.237465,-0.191966,0.225762,0.273348,-0.213611,0.245186,-0.346551,0.332127,-0.213751,-0.187179
82,82,-0.115099,-1.026612,0.152161,0.357461,-0.472162,0.305727,0.179391,0.336356,-0.885303,-0.194595


In [18]:
# get embedding of cat_type

cat_type_model = Model(inputs = embedding_model.get_layer("cat_type").input,\
                outputs = embedding_model.get_layer("cat_type").output)
cat_type_list = []
for x in set(data.cat_type):
    embedding = cat_type_model.predict(np.array([x])).reshape(output_dim_cat_type, )
    cat_type_list.append([x] + list(embedding))

cat_type_columns = ['cat_type']
for i in range(output_dim_cat_type):
    cat_type_columns.append("cat_type_" + str(i))

embed_cat_type = pd.DataFrame(cat_type_list)
embed_cat_type.columns = cat_type_columns
embed_cat_type

,cat_type,cat_type_0,cat_type_1,cat_type_2,cat_type_3
0,0,-0.065210,-0.350239,0.033259,0.362272
1,1,-0.655040,0.796905,-0.577138,0.187011
2,2,1.598249,-1.524056,-1.026803,-1.536040
3,3,1.759115,-1.713067,-1.686697,-1.876777
4,4,-0.754949,1.473959,-0.293846,0.519351
5,5,-0.645446,0.426009,0.679812,0.900362
6,6,0.088758,0.083540,-0.281670,-0.079843
7,7,0.170987,0.445880,-1.915781,-0.893323
8,8,0.633097,-1.201491,-0.087978,-0.270479
9,9,0.701644,-0.795159,-1.674739,-0.958796


In [19]:
# get embedding of sub_type

sub_type_model = Model(inputs = embedding_model.get_layer("sub_type").input,\
                outputs = embedding_model.get_layer("sub_type").output)
sub_type_list = []
for x in set(data.sub_type):
    embedding = sub_type_model.predict(np.array([x])).reshape(output_dim_sub_type, )
    sub_type_list.append([x] + list(embedding))

sub_type_columns = ['sub_type']
for i in range(output_dim_sub_type):
    sub_type_columns.append("sub_type_" + str(i))

embed_sub_type = pd.DataFrame(sub_type_list)
embed_sub_type.columns = sub_type_columns
embed_sub_type

,sub_type,sub_type_0,sub_type_1,sub_type_2,sub_type_3,sub_type_4,sub_type_5,sub_type_6,sub_type_7
0,0,-0.497934,-0.317407,-0.947504,0.239314,0.450626,-0.172902,0.635352,-0.984790
1,1,1.181379,1.300150,2.277320,-0.846083,-1.019895,0.715887,-1.351326,2.533916
2,2,0.437431,0.563644,1.025218,-0.323122,-0.397309,0.268898,-0.626564,1.312198
3,3,0.803415,0.694999,0.756308,-0.696306,-0.808948,0.580435,-0.724860,0.749242
4,4,0.666804,0.822870,0.463541,-0.560998,-0.623881,0.596056,-0.619757,0.257186
...,...,...,...,...,...,...,...,...,...
62,62,1.122412,0.984142,1.112484,-1.121889,-1.226396,1.149887,-1.109049,0.361079
63,63,0.961657,0.668592,0.681390,-0.769049,-0.814706,0.774649,-0.614455,0.033577
64,64,-0.208881,-0.374486,0.097905,0.363624,0.236113,-0.315699,0.369814,0.280479
65,65,0.275299,0.292742,0.199733,-0.211658,-0.344209,0.226128,-0.288610,-0.125368


In [20]:
# get embedding of month

month_model = Model(inputs = embedding_model.get_layer("month").input,\
                outputs = embedding_model.get_layer("month").output)
month_list = []
for x in set(data.month):
    embedding = month_model.predict(np.array([x])).reshape(output_dim_month, )
    month_list.append([x] + list(embedding))

month_columns = ['month']
for i in range(output_dim_month):
    month_columns.append("month_" + str(i))

embed_month = pd.DataFrame(month_list)
embed_month.columns = month_columns
embed_month

,month,month_0,month_1,month_2,month_3
0,0,1.463595,-1.577583,-0.637801,2.121079
1,1,-0.050046,0.375853,0.194426,-0.230493
2,2,-0.017651,0.046792,0.267384,0.282052
3,3,1.736354,-1.912091,-1.317618,2.254421
4,4,0.180297,-0.122407,0.198205,0.465501
5,5,-0.548948,0.670716,0.574580,-0.550619
6,6,0.113088,0.108733,0.029343,0.059785
7,7,-1.239444,1.535462,1.221699,-1.367171
8,8,-0.237488,0.338336,0.316400,-0.075325
9,9,0.132109,-0.063321,0.136406,0.305556


In [21]:
data = pd.merge(data, embed_date_block_num, on=['date_block_num'], how = 'left')
data = pd.merge(data, embed_shop_id, on=['shop_id'], how = 'left')
#data = pd.merge(data, embed_shop_city, on=['shop_city'], how = 'left')
data = pd.merge(data, embed_shop_type, on=['shop_type'], how = 'left')
data = pd.merge(data, embed_item_cat, on=['item_category_id'], how = 'left')
data = pd.merge(data, embed_cat_type, on=['cat_type'], how = 'left')
data = pd.merge(data, embed_sub_type, on=['sub_type'], how = 'left')
data = pd.merge(data, embed_month, on=['month'], how = 'left')

data

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,sub_type_2,sub_type_3,sub_type_4,sub_type_5,sub_type_6,sub_type_7,month_0,month_1,month_2,month_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,1.603588,-0.706945,-0.903334,0.712955,-1.091941,1.659046,1.463595,-1.577583,-0.637801,2.121079
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.517132,0.051013,-0.095740,-0.161716,-0.121499,0.392261,1.463595,-1.577583,-0.637801,2.121079
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,-0.809338,-0.042285,0.143616,0.012080,0.138739,-0.933260,1.463595,-1.577583,-0.637801,2.121079
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,-1.365511,0.197608,0.266568,-0.058177,0.621114,-1.855179,1.463595,-1.577583,-0.637801,2.121079
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,1.603588,-0.706945,-0.903334,0.712955,-1.091941,1.659046,1.463595,-1.577583,-0.637801,2.121079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.463541,-0.560998,-0.623881,0.596056,-0.619757,0.257186,-0.471321,0.530101,0.400101,-0.346533
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.400971,-0.160235,-0.245106,0.256628,-0.364364,-0.594764,-0.471321,0.530101,0.400101,-0.346533
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.463541,-0.560998,-0.623881,0.596056,-0.619757,0.257186,-0.471321,0.530101,0.400101,-0.346533
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,1.603588,-0.706945,-0.903334,0.712955,-1.091941,1.659046,-0.471321,0.530101,0.400101,-0.346533


In [22]:
data.to_pickle('v30_data_embed.pkl')

gc.collect();

In [23]:
print("complete!")

complete!
